In [8]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing
from scipy.stats import t
import matplotlib.pyplot as plt
import math


In [2]:

# 2
data_set = sm.datasets.get_rdataset('airquality')
# df = pd.read_csv('https://dl.dropboxusercontent.com/u/6044937/Data%20Sets/airquality.csv', index_col=0)
# print data_set.data


In [3]:

# 3

df = data_set.data
# data_set.dropna(inplace=True)
df_clean = df.dropna()

print df_clean

     Ozone  Solar.R  Wind  Temp  Month  Day
0       41      190   7.4    67      5    1
1       36      118   8.0    72      5    2
2       12      149  12.6    74      5    3
3       18      313  11.5    62      5    4
6       23      299   8.6    65      5    7
7       19       99  13.8    59      5    8
8        8       19  20.1    61      5    9
11      16      256   9.7    69      5   12
12      11      290   9.2    66      5   13
13      14      274  10.9    68      5   14
14      18       65  13.2    58      5   15
15      14      334  11.5    64      5   16
16      34      307  12.0    66      5   17
17       6       78  18.4    57      5   18
18      30      322  11.5    68      5   19
19      11       44   9.7    62      5   20
20       1        8   9.7    59      5   21
21      11      320  16.6    73      5   22
22       4       25   9.7    61      5   23
23      32       92  12.0    61      5   24
27      23       13  12.0    67      5   28
28      45      252  14.9    81 

In [165]:
# 4
# ozone_train = []
# for data in data_df['Ozone']:
# 	ozone_train.append([data])
    
# solarR_train = []
# for data in data_df['Solar.R']:
# 	solarR_train.append([data])

# ozone_train = df_clean[['Ozone']]
ozone_train = df_clean.ix[:,0]

explVariables_train = df_clean.drop('Ozone', 1)

# Centrée réduit

# ozone_scaled = (ozone_train - np.mean(ozone_train))/np.std(ozone_train)
# explVariables_scaled = (explVariables_train - np.mean(explVariables_train))/np.std(explVariables_train)
ozone_scaled = preprocessing.scale(ozone_train)
explVariables_scaled = preprocessing.scale(explVariables_train)

# Create linear regression object
regr = linear_model.LinearRegression()
# Train the model using the training sets
# regr.fit(explVariables_scaled, ozone_scaled)
regr.fit(explVariables_scaled, ozone_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [166]:
# 5
coef = regr.coef_
intercept = regr.intercept_

print 'Coef: ' + str(coef)
print 'Intercept: ' + str(intercept)

ozoneP = regr.predict(explVariables_scaled)

# print 'ozonePredict: ' + str(ozoneP)

# residual = np.subtract(ozone_scaled, ozoneP)
residual = np.subtract(ozone_train, ozoneP)


# print np.sqrt(np.sum(residual*residual))
# matrix rank
p = np.linalg.matrix_rank(explVariables_scaled)
# number of explicative variable
n=len(residual)
                          
sigmaCarre = (np.sum(residual*residual) / (n - p - 1))
print np.sqrt(sigmaCarre)
print 'Sigma carre: ' + str(sigmaCarre)

Coef: [  4.56193076 -11.75277084  17.98521995  -4.45895297   2.37393859]
Intercept: 42.0990990991
20.85846339
Sigma carre: 435.075494992


In [171]:
# 6
alpha = 0.01

# IC = Regression_data_clean_toOzone_Coef[
#    0] - stud[0] * sqrt(EstimateurSansBiaisVar) * sqrt(AllExeptOzone_x[['Solar.R']].transpose()*AllExeptOzone_x['Solar.R'])
interStud = t.interval(1-alpha, n - p - 1)

# print interStud

# IC = coef[0] - stud[0] * np.sqrt(sigmaCarre) * np.sqrt(explVariables_scaled[0].transpose()*explVariables_scaled[0])
XtX = np.dot(explVariables_scaled.transpose(),explVariables_scaled)
XtXinv = np.linalg.inv(XtX)

# ICbornMinus = coef[0] - interStud[0] * np.sqrt(sigmaCarre) * np.sqrt( XtXinv[0][0])
# ICbornMax = coef[0] + interStud[0] * np.sqrt(sigmaCarre) * np.sqrt(XtXinv[0][0])

# ICbornMinus = coef[0] - interStud[0] * np.sqrt(sigmaCarre) * np.sqrt( 1.0/XtX[0][0])
# ICbornMax = coef[0] + interStud[0] * np.sqrt(sigmaCarre) * np.sqrt(1.0/XtX[0][0])

# print XtXinv[0][0]
# print 1.0/XtX[0][0]

ICbornMinus = []
ICbornMax = []

for i in range(5):
    ICbornMinus.append(coef[i] - interStud[0] * np.sqrt(sigmaCarre) * np.sqrt(XtXinv[0][0])) 
    ICbornMax.append(coef[i] + interStud[0] * np.sqrt(sigmaCarre) * np.sqrt(XtXinv[0][0])) 


    
    
    
for i in range(5):
    print 'Theta'+str(i+1)+': ' + str(coef[i]) + '       Confidence Interval: [' + str(ICbornMax[i]) +'; ' + str(ICbornMinus[i]) + ']'


Theta1: 4.56193075918       Confidence Interval: [-1.01299188923; 10.1368534076]
Theta2: -11.7527708448       Confidence Interval: [-17.3276934932; -6.17784819636]
Theta3: 17.9852199495       Confidence Interval: [12.4102973011; 23.5601425979]
Theta4: -4.45895296698       Confidence Interval: [-10.0338756154; 1.11596968143]
Theta5: 2.37393858761       Confidence Interval: [-3.2009840608; 7.94886123602]


In [ ]:
# 7



In [182]:
# 8

newObs = np.array([197, 10, 70, 1, 3], float)

varY = np.var(ozone_train)
varX = []
for i in range(5):
    xi_train = df_clean.ix[:,i+1]
    varX.append(np.var(xi_train))

# print varY
# print varX

thetaUnscaled = []
ICbornMinusUnscaled = []
ICbornMaxUnscaled = []
for i in range(5):
    thetaUnscaled.append((coef[i] * varY)/varX[i])
    ICbornMinusUnscaled.append((ICbornMinus[i] * varY)/varX[i])
    ICbornMaxUnscaled.append((ICbornMax[i] * varY)/varX[i])
    
bornMinusPred = np.sum(ICbornMinusUnscaled)
bornMaxPred = np.sum(ICbornMaxUnscaled)
thetaPred = np.sum(thetaUnscaled) + intercept

print 'Ozone'+': ' + str(thetaPred) + '       Confidence Interval: [' + str(bornMaxPred) +'; ' + str(bornMinusPred) + ']'



Ozone: -3005.72439274       Confidence Interval: [-6529.07170496; 433.424721284]
